In [1]:
#%run prep.py -d accounts

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

In [3]:
import dask.bag as db

b = db.from_sequence(range(1, 11), npartitions=2)

b.take(3)

(1, 2, 3)

In [4]:
import os

b = db.read_text(os.path.join("data", "accounts.*.json.gz"))

b.take(1)

('{"id": 0, "name": "Jerry", "transactions": [{"transaction-id": 1432, "amount": -523}, {"transaction-id": 2612, "amount": -571}, {"transaction-id": 2827, "amount": -452}, {"transaction-id": 5629, "amount": -366}, {"transaction-id": 9471, "amount": -556}, {"transaction-id": 11263, "amount": -370}, {"transaction-id": 13202, "amount": -535}, {"transaction-id": 14831, "amount": -286}, {"transaction-id": 16904, "amount": -262}, {"transaction-id": 17963, "amount": -387}, {"transaction-id": 20716, "amount": -233}, {"transaction-id": 21943, "amount": -242}, {"transaction-id": 22308, "amount": -316}, {"transaction-id": 24801, "amount": -413}, {"transaction-id": 25090, "amount": -372}, {"transaction-id": 25257, "amount": -559}, {"transaction-id": 25336, "amount": -579}, {"transaction-id": 28224, "amount": -168}, {"transaction-id": 29786, "amount": -156}, {"transaction-id": 30079, "amount": -584}, {"transaction-id": 31388, "amount": -447}, {"transaction-id": 33140, "amount": -403}, {"transaction

In [5]:
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [7]:
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})

b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

In [8]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence(range(1, 11))

c = b.filter(is_even).map(lambda x: x ** 2)

c

dask.bag<lambda, npartitions=10>

In [9]:
c.compute()

[4, 16, 36, 64, 100]

In [10]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Jerry", "transactions": [{"transaction-id": 1432, "amount": -523}, {"transaction-id": 2612, "amount": -571}, {"transaction-id": 2827, "amount": -452}, {"transaction-id": 5629, "amount": -366}, {"transaction-id": 9471, "amount": -556}, {"transaction-id": 11263, "amount": -370}, {"transaction-id": 13202, "amount": -535}, {"transaction-id": 14831, "amount": -286}, {"transaction-id": 16904, "amount": -262}, {"transaction-id": 17963, "amount": -387}, {"transaction-id": 20716, "amount": -233}, {"transaction-id": 21943, "amount": -242}, {"transaction-id": 22308, "amount": -316}, {"transaction-id": 24801, "amount": -413}, {"transaction-id": 25090, "amount": -372}, {"transaction-id": 25257, "amount": -559}, {"transaction-id": 25336, "amount": -579}, {"transaction-id": 28224, "amount": -168}, {"transaction-id": 29786, "amount": -156}, {"transaction-id": 30079, "amount": -584}, {"transaction-id": 31388, "amount": -447}, {"transaction-id": 33140, "amount": -403}, {"transaction

In [11]:
import json

js = lines.map(json.loads)

js.take(3)

({'id': 0,
  'name': 'Jerry',
  'transactions': [{'transaction-id': 1432, 'amount': -523},
   {'transaction-id': 2612, 'amount': -571},
   {'transaction-id': 2827, 'amount': -452},
   {'transaction-id': 5629, 'amount': -366},
   {'transaction-id': 9471, 'amount': -556},
   {'transaction-id': 11263, 'amount': -370},
   {'transaction-id': 13202, 'amount': -535},
   {'transaction-id': 14831, 'amount': -286},
   {'transaction-id': 16904, 'amount': -262},
   {'transaction-id': 17963, 'amount': -387},
   {'transaction-id': 20716, 'amount': -233},
   {'transaction-id': 21943, 'amount': -242},
   {'transaction-id': 22308, 'amount': -316},
   {'transaction-id': 24801, 'amount': -413},
   {'transaction-id': 25090, 'amount': -372},
   {'transaction-id': 25257, 'amount': -559},
   {'transaction-id': 25336, 'amount': -579},
   {'transaction-id': 28224, 'amount': -168},
   {'transaction-id': 29786, 'amount': -156},
   {'transaction-id': 30079, 'amount': -584},
   {'transaction-id': 31388, 'amount': 

In [13]:
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 17,
  'name': 'Alice',
  'transactions': [{'transaction-id': 786, 'amount': 555},
   {'transaction-id': 1294, 'amount': 444},
   {'transaction-id': 1759, 'amount': 469},
   {'transaction-id': 2434, 'amount': 458},
   {'transaction-id': 3016, 'amount': 545},
   {'transaction-id': 3179, 'amount': 532},
   {'transaction-id': 4566, 'amount': 513},
   {'transaction-id': 4648, 'amount': 487},
   {'transaction-id': 4969, 'amount': 520},
   {'transaction-id': 5156, 'amount': 511},
   {'transaction-id': 5181, 'amount': 516},
   {'transaction-id': 5194, 'amount': 429},
   {'transaction-id': 5218, 'amount': 474},
   {'transaction-id': 5421, 'amount': 427},
   {'transaction-id': 5645, 'amount': 582},
   {'transaction-id': 6025, 'amount': 537},
   {'transaction-id': 6538, 'amount': 537},
   {'transaction-id': 6958, 'amount': 449},
   {'transaction-id': 7200, 'amount': 468},
   {'transaction-id': 7872, 'amount': 489},
   {'transaction-id': 8794, 'amount': 471},
   {'transaction-id': 9299, 'a

In [14]:
def count_transactions(d):
    return {
        'name': d['name'],
        'count': len(d['transactions'])
    }

(js.filter(lambda record: record['name'] == 'Alice')
    .map(count_transactions)
    .take(5))

({'name': 'Alice', 'count': 207},
 {'name': 'Alice', 'count': 91},
 {'name': 'Alice', 'count': 419},
 {'name': 'Alice', 'count': 516},
 {'name': 'Alice', 'count': 98})

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
    .map(count_transactions)
    .pluck('count')
    .mean()
    .compute())

196.93666666666667

In [17]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Jerry,"[{'transaction-id': 1432, 'amount': -523}, {'t..."
1,1,Jerry,"[{'transaction-id': 1345, 'amount': 4511}, {'t..."
2,2,Patricia,"[{'transaction-id': 31, 'amount': 226}, {'tran..."
3,3,Ray,"[{'transaction-id': 562, 'amount': 238}, {'tra..."
4,4,Edith,"[{'transaction-id': 5849, 'amount': 2424}, {'t..."


In [18]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 325 ms, sys: 48 ms, total: 373 ms
Wall time: 2.12 s


name
Alice      900
Bob        932
Charlie    899
Dan        850
Edith      800
Name: id, dtype: int64

In [19]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'],
             'name': record['name'],
             'amount': transaction['amount'],
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Jerry', 'amount': -523, 'transaction-id': 1432},
 {'id': 0, 'name': 'Jerry', 'amount': -571, 'transaction-id': 2612},
 {'id': 0, 'name': 'Jerry', 'amount': -452, 'transaction-id': 2827})

In [20]:
df = transactions.to_dataframe()

df.head()

,id,name,amount,transaction-id
0,0,Jerry,-523,1432
1,0,Jerry,-571,2612
2,0,Jerry,-452,2827
3,0,Jerry,-366,5629
4,0,Jerry,-556,9471


In [21]:
df.groupby('name')['transaction-id'].count().compute()

name
Alice       177243
Bob         215686
Charlie     227750
Dan         144982
Edith       199663
Frank       249985
George       83809
Hannah      214262
Ingrid      162715
Jerry       271400
Kevin       254336
Laura        78895
Michael     176207
Norbert     125048
Oliver      114303
Patricia    394748
Quinn       225997
Ray         221710
Sarah       157858
Tim         199723
Ursula      179528
Victor      180573
Wendy       252145
Xavier      192703
Yvonne      113035
Zelda       185696
Name: transaction-id, dtype: int64

In [22]:
client.shutdown()